# Yelp Restaurant Rating Prediction

## Project Description
The project goal is to predict restaurant overall ratings on yelp
in New York City, using multiple features of restaurants that we will
extract using the yelp API. Our motivation is to help determine how
successful a new restaurant business might be, given certain known
characteristics of it.

We will use the Yelp API, with the help of pandas, to acquire raw data
from restaurants. Then we will extract reasonable features such as
location, open hours, whether it takes reservations, whether it has
delivery service, whether there is parking space, and whether it
provides free wifi etc., from the parsed data, and combine with the
overall ratings, which is a numerical value ranging from 0 to 5, as
labels.

We will model the rating distribution over the different features that
we extract and create, and analyse how much each feature shifts our
distribution. Using our results from this we will select good features
to train on machine learning models.

Using the labeled features that we construct, we will train different
machine learning models like linear regression, nonlinear regression,
logistic regression as well as neural networks, then make some
predictions, and compare the accuracy obtained from them.

## Team Members
Jun Hee Kim, Nikhil Rangarajan, Sander Shi

## Procedure
* [Data Gathering from API](#step-1)
* [Feature Extraction with Parsing](#step-2)
* [Feature Analysis and Variable Selection](#step-3)
* [Setup of Models](#step-4)
* [Cross Validation](#step-5)
* [Final Analysis](#step-6)

## Part 0: Imports and Definitions of Constants

We will be using `pandas` to parse the data and `tensorflow` to construct the machine learning models. We will also be using the Yelp API to gather the data. In order to
use the Yelp API, we need to create an API key to use in our API requests.

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import requests
import json
import time
import pickle

API_URL = "https://api.yelp.com/v3/businesses"
SEARCH_URL = API_URL + "/search"
RESTAURANT_IDS = "rid.pkl"

with open("./API_KEY", 'r') as f:
    api_key = f.readline().strip()
API_HEADERS = {
    'Authorization': ' '.join(['Bearer', api_key])
}

<a id="step-1"></a>

## Part 1: Data Gathering from API

In this step we will use the Yelp API to gather restaurant pages, then extract
information using business search API requests.

Since the Yelp API will only allow us to load business information from up to 1000
distinct businesses on each search, we will first get the categories of the
restaurants, then perform a search query for each category.

In [2]:
def get_restaurant_categories(url):
    """
    This function gets all restaurant categories.
    
    @input url: URL to json file containing restaurant categories.
    @type url: String.
    
    @return: List of restaurant categories.
    @rtype: List of String
    """
    cats = json.load(open(url))
    return [cat['alias'] for cat in cats if 'restaurants' in cat['parents']]

categories = get_restaurant_categories('categories.json')

Now that we have the restaurant categories, we can write a function to find all
restaurant IDs, and create a Pandas DataFrame. To aid debugging, we will dump
the dataframe into a pickle file so that we will not have to regenerate each time.

In [3]:
def get_req_json(url, params=None):
    response = requests.get(url=url, headers=API_HEADERS, params=params)
    return response.json()


def find_restaurants(url, categories):
    """
    This function loads all restaurant data from restaurants in Pittsburgh.
    
    @input url: The API url.
    @type url: String.
    @input categories: The restaurant categories.
    @type categories: List of String.
    
    @return: A Pandas DataFrame containing the restaurant URLs.
    @rtype: pandas.DataFrame.
    """
    restaurants = []
    for cat in categories:
        params = {
            'term': 'restaurants',
            'location': 'NYC',
            'categories': cat
        }
        content = get_req_json(url, params)
        total = min(1000, content['total'])
        i = 0
        while i < total:
            limit = min(50, total - i)
            params['limit'] = limit
            params['offset'] = i
            content = get_req_json(url, params)
            restaurants.extend([b['id'] for b in content['businesses']])
            i += limit
    return pd.DataFrame({'id': list(set(restaurants))})

restaurants = find_restaurants(SEARCH_URL, categories)
pickle.dump(restaurants, open(RESTAURANT_IDS, "wb"))

Stuff

In [6]:
def get_features(rids, api_url):
    prices, ratings, pickups, deliveries= [], [], [], []
    reservations, is_claimed, zipcodes = [], [], []
    for idx, rid in rids.iterrows():
        url = "/".join([api_url, rid['id']])
        content = get_req_json(url)
        prices.append(len(content['price']) if 'price' in content else 0)
        ratings.append(content['rating'] if 'rating' in content else 0)
        pickups.append("pickup" in content['transactions'])
        deliveries.append("delivery" in content['transactions'])
        reservations.append("restaurant_reservation" in content['transactions'])
        is_claimed.append(content['is_claimed'])
        zipcodes.append(content['location']['zip_code'])
    df_dict = {
        'Price': prices,
        'Pickup': pickups,
        'Delivery': deliveries,
        'Reservation': reservations,
        'Claimed': is_claimed,
        'Zipcode': zipcodes,
        'Ratings': ratings
    }
    return pd.DataFrame(df_dict)

# restaurants = find_restaurants(SEARCH_URL, categories)
# pickle.dump(restaurants, open(RESTAURANT_IDS, "wb"))

restaurant_ids = pickle.load(open(RESTAURANT_IDS, 'rb'))
restaurant_ids
# labeled_features = get_features(restaurant_ids, API_URL)
# pickle.dump(labeled_features, open("features.pkl", "wb"))
# labeled_features = pickle.load(open("features.pkl", "rb"))
# print(len(labeled_features))
# labeled_features

,id
0,UTNSUIY2QW6RYNa0xLa9gg
1,h5WkewI6U7NjLB_n6WgdvQ
2,yOPbrFZ2ZUsyQWsedP6gUg
3,2ngodDyMj6M0szyBbGSJ5g
4,8ghGdEWOkGWUNeGrtdiaDQ
5,58v6bJy_vQLMWxLSMZHvYA
6,4LoroCI3VtQ17dyQa75-Rg
7,1_JZHKkWQBozYQ00fephWQ
8,YRfDdBgWS4T80oJfKmVThA
9,IUMTe7g19htqG7MXrsP5NQ


<a id="step-2"></a>

## Part 2: Feature Extraction with Parsing

In this step we will parse the raw features we got from the API and mold them into
discrete and continuous values.

In [3]:
def extract_features(raw_data):
    """
    This function extracts the features from raw restaurant data.
    """
    pass

labeled_features = extract_features(all_restaurants)

<a id="step-3"></a>

## Part 3: Feature Analysis and Variable Selection

Now that we have the features, we can separately analyse them and see how each feature
contributes to the rating.

In [4]:
# TODO: Jun Hee's Feature Analysis

<a id="step-4"></a>

## Part 4: Setup of Models

We need to construct a couple machine learning models for us to train to see which
one gives us the best testing accuracy.

In [5]:
class ANN(object):
    """
    This is a neural network.
    """
    def __init__(self, layers=[5, 5]):
        self.layers = layers
        
class LogisticRegression(object):
    """
    This is the logistic regression model.
    """
    def __init__(self):
        self.theta = np.zeros(100)

<a id="step-5"></a>

## Part 5: Cross Validation

Now we will use cross-validation to determine which model produces the highest
validation accuracy for our dataset, then pick this model.

In [6]:
def pick_model():
    pass

<a id="step-6"></a>

## Part 6: Final Analysis

Our results were amazing, so yeah.